In [1]:
import sys
sys.path.append('/Users/mnannan/dev/supelec/ngsa/assignments/Assignement 2 ')

In [2]:
from main import pipeline
from core.io.loader import *
from features.collaboration import *
from features.papers_graph import *
from features.authors import *

In [3]:
from features.title import *

In [4]:
from sklearn.metrics.pairwise import cosine_similarity

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [6]:
import time

In [7]:
vectorizer = TfidfVectorizer(analyzer='word', min_df=0, stop_words="english", ngram_range=(1, 3))

In [8]:
train = get_data_with_node_information('train', data_dir='../data')

In [9]:
papers = train[['source_id', 'source_title', 'source_abstract', 'source_journal']]\
.drop_duplicates('source_id').reset_index(drop=True)

In [10]:
paper_id_mapping = {source_id: index for index, source_id in list(papers.source_id.items())}

In [11]:
source_ids = train.source_id.apply(lambda x: paper_id_mapping[x])

In [12]:
target_ids = train.target_id.apply(lambda x: paper_id_mapping[x])

###### title 

In [10]:
title_tfidf = vectorizer.fit_transform(papers.source_title.values)

In [37]:
title_cosimilarity_matrix = cosine_similarity(title_tfidf, title_tfidf)

6.415889263153076


In [59]:
train['title_cosimilarity'] = title_cosimilarity_matrix[source_ids, target_ids]

###### abstract

In [38]:
abstract_tfidf = vectorizer.fit_transform(papers.source_abstract.values)

In [39]:
abstract_cosimilarity_matrix = cosine_similarity(abstract_tfidf, abstract_tfidf)

51.42162895202637


In [63]:
train['abstract_cosimilarity'] = abstract_cosimilarity_matrix[source_ids, target_ids]

###### journal

In [65]:
journal_vectorizer = TfidfVectorizer(analyzer='word',
                                     min_df=0,
                                     stop_words="english",
                                     tokenizer = lambda x: x.split('.'),
                                     ngram_range=(1, 3))

In [67]:
journal_tfidf = journal_vectorizer.fit_transform(papers.source_journal.fillna('').values)

In [69]:
journal_cosimilarity_matrix = cosine_similarity(journal_tfidf, journal_tfidf)

In [70]:
train['journal_cosimilarity'] = journal_cosimilarity_matrix[source_ids, target_ids]

###### Journal x abstract

In [13]:
nb_papers = papers.shape[0]

In [36]:
title_abstract_vectorizer = TfidfVectorizer(analyzer='word', min_df=0, stop_words="english", ngram_range=(1, 3))

In [37]:
title_abstract_tfidf = title_abstract_vectorizer.fit(pd.concat([papers.source_title, papers.source_abstract]).values)

In [39]:
title_abstract_cosimilarity_matrix = \
    cosine_similarity(title_abstract_tfidf.transform(papers.source_title.values),
                      title_abstract_tfidf.transform(papers.source_abstract.values))

In [40]:
title_abstract_cosimilarity_matrix.shape

(27770, 27770)

In [23]:
title_abstract_cosimilarity_matrix = title_abstract_cosimilarity_matrix

In [41]:
train['source_title_target_abstract_cosimilarity'] = title_abstract_cosimilarity_matrix[source_ids, target_ids]

train['source_abstract_target_title_cosimilarity'] = title_abstract_cosimilarity_matrix[target_ids, source_ids]